# DFT y DFT Inversa

## Conceptos Clave

Según la Clase 12, la Transformada Discreta de Fourier (DFT) y su inversa (IDFT) son herramientas fundamentales para analizar señales en el dominio de la frecuencia discreta y para reconstruir la señal original a partir de su representación en frecuencia.

La **DFT de N puntos** de una secuencia $x(n)$ de longitud $N$ se define como[cite: 25]:
$$X(k) = \sum_{n=0}^{N-1} x(n) e^{-j 2\pi kn/N}, \quad \text{para } k=0, 1, \dots, N-1$$
donde $X(k)$ son los coeficientes de la DFT en el dominio de la frecuencia.

La **DFT Inversa (IDFT)** permite obtener la secuencia original $x(n)$ a partir de sus coeficientes de la DFT $X(k)$[cite: 35]:
$$x(n) = \frac{1}{N} \sum_{k=0}^{N-1} X(k) e^{j 2\pi kn/N}, \quad \text{para } n=0, 1, \dots, N-1$$

Estas transformadas permiten pasar del dominio del tiempo discreto al dominio de la frecuencia discreta y viceversa.

## Ejemplo 1: Cálculo de la DFT Inversa (IDFT)

Este ejemplo reproduce el cálculo de la IDFT mostrado en la Clase 12[cite: 37]. Dada una secuencia de coeficientes de la DFT $X(k)$, calcularemos la secuencia original en el dominio del tiempo $x(n)$ utilizando la fórmula de la IDFT.

**Secuencia de la DFT dada:**
$X = [6, 8+4j, -2, 8-4j]$

**Tamaño de la transformada (N):**
El número de puntos es igual a la longitud de la secuencia $X$, que es $N=4$.

Según la Clase 12, al aplicar la IDFT a esta secuencia, se obtiene la secuencia $x = [5, 0, -3, 4]$[cite: 37]. Verificaremos esto usando NumPy.

### Código Python para Ejemplo 1


In [1]:
# --------------- Imports ---------------
import numpy as np

# --- Ejemplo 1: Cálculo de la IDFT ---
print("--- Ejemplo 1: Cálculo de la IDFT ---")

# Secuencia de la DFT (X[k])
X = np.array([6, 8 + 4j, -2, 8 - 4j])
N = len(X) # Número de puntos de la transformada

print(f"Secuencia de la DFT (X): {X}")
print(f"Número de puntos (N): {N}")

# Calcular la IDFT usando numpy.ifft
# numpy.ifft ya incluye el factor 1/N y usa la exponencial compleja positiva
x_calculated = np.fft.ifft(X)

print(f"Secuencia calculada en el dominio del tiempo (x): {x_calculated}")

# El resultado puede tener una parte imaginaria muy pequeña debido a errores de punto flotante.
# La parte imaginaria suele ser despreciable si la señal original era real.
x_real = np.real(x_calculated)

print(f"Parte real de la secuencia calculada (x_real): {x_real}")

# Resultado esperado según la Clase 12: x = [5, 0, -3, 4]
x_expected = np.array([5, 0, -3, 4])

# Comparar el resultado calculado con el esperado (permitiendo una pequeña tolerancia)
are_close = np.allclose(x_real, x_expected)

print(f"¿El resultado calculado coincide con el esperado? {are_close}")
if are_close:
    print("El cálculo de la IDFT coincide con el ejemplo de la Clase 12.")
else:
    print("El cálculo de la IDFT NO coincide exactamente con el ejemplo de la Clase 12 (puede haber diferencias mínimas por punto flotante).")

print("\n--- Fin del Ejemplo 1 ---\n")

--- Ejemplo 1: Cálculo de la IDFT ---
Secuencia de la DFT (X): [ 6.+0.j  8.+4.j -2.+0.j  8.-4.j]
Número de puntos (N): 4
Secuencia calculada en el dominio del tiempo (x): [ 5.+0.j  0.+0.j -3.+0.j  4.+0.j]
Parte real de la secuencia calculada (x_real): [ 5.  0. -3.  4.]
¿El resultado calculado coincide con el esperado? True
El cálculo de la IDFT coincide con el ejemplo de la Clase 12.

--- Fin del Ejemplo 1 ---



## Ejemplo Extra: Visualización Interactiva de la DFT

Este ejemplo te permite visualizar la Transformada Discreta de Fourier (DFT) de diferentes tipos de señales simples en el dominio del tiempo y su correspondiente espectro de magnitud en el dominio de la frecuencia. Puedes ajustar el número de puntos (<span class="math-inline">N</span>) de la transformada para observar cómo afecta la representación en frecuencia.

Al cambiar el número de puntos <span class="math-inline">N</span>, estás cambiando cuántas muestras de la señal se incluyen en la transformada y el número de puntos en el dominio de la frecuencia en los que se evalúa la DFT. Esto influye en la **resolución en frecuencia** (<span class="math-inline">f\_s/N</span>).

Explora cómo se ven en el dominio de la frecuencia:
* Un impulso (delta de Dirac discreta).
* Un Coseno.
* Una onda cuadrada.


In [ ]:
# --------------- Imports ---------------
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import IntSlider, Dropdown, VBox, interactive_output, Layout
from IPython.display import display, clear_output
import math

# --- Función para calcular y graficar la DFT ---

def calculate_and_plot_dft(signal_type, N):
    """
    Genera una señal de tiempo discreto, calcula su DFT y grafica ambos.
    """
    # Limpiar la salida anterior
    with output_area:
        clear_output(wait=True)

    print(f"--- Señal: {signal_type}, N = {N} puntos ---")

    # Vector de tiempo discreto
    n = np.arange(N)

    # Generar la señal en el dominio del tiempo x(n)
    if signal_type == "Impulso (Delta)":
        x = np.zeros(N)
        if N > 0:
            x[0] = 1 # Impulso en n=0
        signal_title = r"Impulso Unitario $\delta(n)$" # Usar raw string para LaTeX
    elif signal_type == "Coseno":
        # Generar un coseno con una frecuencia que resulte en un número entero de ciclos en N puntos
        # Esto ayuda a ver picos limpios en la DFT sin "leakage"
        freq_cycles = 4 # Número de ciclos dentro de N puntos
        # Asegurarse de que N sea suficiente para al menos 1 ciclo completo si freq_cycles > 0
        if N > 0 and freq_cycles > 0 and N < N / freq_cycles : # Correction check if N is too small for freq_cycles
             freq_cycles = 1 # Default to 1 cycle if N is too small
        x = np.cos(2 * np.pi * freq_cycles * n / N)
        signal_title = f"Coseno ({freq_cycles} ciclos en N={N} pts)" # No LaTeX aquí
    elif signal_type == "Onda Cuadrada":
        # Generar una onda cuadrada simple
        x = np.zeros(N)
        if N > 0:
             x[:N//2] = 1
             x[N//2:] = -1
        signal_title = "Onda Cuadrada" # No LaTeX aquí
    else:
        x = np.zeros(N)
        signal_title = "Seleccione una señal" # No LaTeX aquí


    # --- Calcular la DFT ---
    if N > 0:
        X = np.fft.fft(x)
        # Calcular la magnitud del espectro
        X_mag = np.abs(X)
        # Crear el vector de frecuencias k
        k = np.arange(N)
    else:
        X_mag = np.array([])
        k = np.array([])


    # --- Creación de Subplots ---
    fig, axes = plt.subplots(2, 1, figsize=(10, 7))
    fig.suptitle(f'{signal_title} y su Espectro de Magnitud DFT', fontsize=14) # signal_title handled by raw string if needed

    # Plot 1: Señal en el Tiempo
    # Usar raw string para el título del eje Y si contiene LaTeX
    # Eliminamos 'use_line_collection=True'
    axes[0].stem(n, x, basefmt=" ")
    axes[0].set_title(r"Señal en el Dominio del Tiempo $x(n)$") # Usar raw string para LaTeX
    axes[0].set_xlabel("n (muestra)")
    axes[0].set_ylabel("Amplitud")
    axes[0].grid(True)
    axes[0].axhline(0, color='black', linewidth=0.5)
    axes[0].axvline(0, color='black', linewidth=0.5)


    # Plot 2: Magnitud del Espectro DFT
    if N > 0:
        # Eliminamos 'use_line_collection=True'
        axes[1].stem(k, X_mag, basefmt=" ")
        axes[1].set_title(r"Espectro de Magnitud DFT $|X(k)|$") # Usar raw string para LaTeX si es necesario, aunque |X(k)| suele funcionar
        axes[1].set_xlabel("k (índice de frecuencia)")
        axes[1].set_ylabel("Magnitud")
        axes[1].grid(True)
        axes[1].axhline(0, color='black', linewidth=0.5)
        axes[1].set_xlim(-1, N) # Asegurar que se vean todos los puntos k
    else:
         axes[1].set_title(r"Espectro de Magnitud DFT $|X(k)|$")
         axes[1].set_xlabel("k (índice de frecuencia)")
         axes[1].set_ylabel("Magnitud")
         axes[1].text(0.5, 0.5, 'N debe ser > 0', horizontalalignment='center', verticalalignment='center', transform=axes[1].transAxes, fontsize=12)


    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Ajustar layout

    # Mostrar el gráfico dentro del contexto del output_area
    with output_area:
        plt.show()

    print("\n--- Fin del Gráfico ---\n")


# --- Widgets de Control ---
style = {'description_width': 'initial'}
layout_widget = Layout(width='80%')

signal_dropdown = Dropdown(
    options=["Impulso (Delta)", "Coseno", "Onda Cuadrada"],
    value="Coseno", # Valor por defecto
    description='Tipo de Señal:',
    style=style,
    layout=layout_widget
)

N_slider = IntSlider(
    min=4, max=128, step=4, value=32, # N debe ser al menos 4 para el ejemplo
    description='Número de Puntos (N):',
    style=style,
    layout=layout_widget
)


# --- Contenedor de Salida ---
output_area = widgets.Output()

# --- Conectar Widgets y Mostrar ---
interactive_plot = interactive_output(
    calculate_and_plot_dft,
    {'signal_type': signal_dropdown, 'N': N_slider}
)

print("Ajusta los controles para visualizar la DFT de diferentes señales:")

display(VBox([signal_dropdown, N_slider, output_area]))

# Ejecutar la función inicialmente para mostrar el estado por defecto
with output_area:
     calculate_and_plot_dft(signal_dropdown.value, N_slider.value)

print("\n--- Fin del Código Interactivo ---\n")

Ajusta los controles para visualizar la DFT de diferentes señales:



--- Fin del Código Interactivo ---

